In [ ]:
#!pip install git+https://github.com/brianmanderson/PyESAPI

In [1]:
import pyesapi
import atexit
import SimpleITK as sitk
import os
import numpy as np

In [33]:
def get_direction(varian_image):
    x_direction = [varian_image.get_XDirection()[i] for i in range(3)]
    y_direction = [varian_image.get_YDirection()[i] for i in range(3)]
    z_direction = [varian_image.get_ZDirection()[i] for i in range(3)]
    direction = x_direction + y_direction + z_direction
    return tuple(direction)

def get_spacing(varian_image):
    return (varian_image.get_XRes(), varian_image.get_YRes(), varian_image.get_ZRes())

def get_origin(potential_image):
    return tuple([potential_image.get_Origin()[i] for i in range(3)])

In [34]:
def create_sitk_Image(varian_image):
    image_handle = sitk.GetImageFromArray(varian_image.np_array_like().transpose())
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle

In [43]:
def create_sitk_Mask(varian_mask, varian_image):
    image_handle = sitk.GetImageFromArray(varian_mask.np_mask_like(varian_image).transpose())
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle

In [2]:
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose)

<bound method 'Dispose'>

In [3]:
#for pat_sum in app.PatientSummaries:
#    print(pat_sum.Id, pat_sum.LastName, pat_sum.FirstName)

In [4]:
fid = open(os.path.join('.', 'PatientID'))
line = fid.readline()
line = fid.readline() # Go to the second patient
fid.close()
MRN = line.strip('\n')

In [5]:
patient = app.OpenPatientById(MRN)

In [6]:
courses = [c.Id for c in patient.Courses]

In [7]:
plan = patient.CoursesLot(courses[1]).PlanSetupsLot(0)

In [32]:
type(plan.Dose)

VMS.TPS.Common.Model.API.PlanningItemDose

In [8]:
print(f'Plan Id: {plan.Id}')
print(f'Dose Per Fx: {plan.PrescribedDosePerFraction}')
print(f'Number of Fx: {plan.NumberOfFractions}')

Plan Id: 1 Abdomen
Dose Per Fx: 300.0 cGy
Number of Fx: 10


In [54]:
beams[0].Dose == plan.Dose

False

In [29]:
beams[0].Dose.get_ZRes()

2.5

In [ ]:
print(f'StructureID,TYPE,VOLUME')
for structure in plan.StructureSet.Structures:
    print(f'{structure.Id},{structure.DicomType},{structure.Volume:.2f}')

In [ ]:
image_handle = create_sitk_Image(plan.StructureSet.Image)

In [ ]:
sitk.WriteImage(image_handle, r'K:\Image.nii.gz')

In [44]:
mask_handle = create_sitk_Mask(plan.StructureSet.StructuresLot('Liver'), plan.StructureSet.Image)

In [ ]:
sitk.WriteImage(mask_handle, r'K:\Mask.nii.gz')

In [45]:
dose_handle = create_sitk_Image(plan.Dose)

In [46]:
sitk.WriteImage(dose_handle, r'K:\Dose.nii.gz')

In [49]:
beam_handle = create_sitk_Image(beams[0].Dose)
sitk.WriteImage(dose_handle, r'K:\{}.nii.gz'.format(beams[0].Id))

In [ ]:
plan.StructureSet.StructuresLot('Liver').Volume # cc